<a href="https://colab.research.google.com/github/Sanowarsunny/Thesis-/blob/main/Final_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# **Data Process**

In [ ]:
da = pd.read_csv("/content/drive/MyDrive/thesis/project/Gsesir_Copy.csv")
da

In [ ]:
da.dtypes

In [ ]:
da.describe()

In [ ]:
da.shape

In [ ]:
Y = da['death']
X = da[[i for i in da.columns if i!='death' and da[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
X.columns

In [ ]:
# pandas count distinct values in column
da['death'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.2,random_state = 42)

# **Accuracy**

LR

In [ ]:
lr = LogisticRegression(solver='newton-cg',multi_class='ovr',max_iter=100, fit_intercept= True)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

XGBClassifier

In [ ]:
xg = XGBClassifier(eta = 0.1,
	 max_depth = 10,
	 min_child_weight = 15,
	 n_estimators = 100,
	 n_jobs = 1)

xg.fit(X_train,y_train)
xg.score(X_test, y_test)

RANDOM Froest

In [ ]:
rf = RandomForestClassifier(n_estimators=5)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

DT

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.score(X_test, y_test)

SVM

In [ ]:
svm = SVC(gamma='scale')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

# **feature selection using SHAP**

**XGBoost**

In [ ]:
model = XGBRegressor(n_estimators=1000, max_depth=10, learning_rate=0.001)
# Fit the Model
model.fit(X, Y)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, features=X, feature_names=X.columns)


In [ ]:
# top 10 important features
imp=da[["Age", "Intubation", "rate Po2", "Other chronic diseases",
       "Condition when entering the hospital", "hospital duration", "Cough",
       "myalgia or fatigue", "Shortness of breath",
       "Fever","gender","death"]]
imp


In [ ]:
xgb_imp_out = imp['death']
xgb_imp_input = imp[[i for i in imp.columns if i!='death' and imp[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
xgb_imp_input.describe

In [ ]:
xg = XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.001)
# Fit the Model
xg.fit( xgb_imp_input, xgb_imp_out)
xg.score( xgb_imp_input, xgb_imp_out)

In [ ]:
from sklearn.model_selection import train_test_split
im_train, im_test, out_train, out_test = train_test_split(
    xgb_imp_input,
     xgb_imp_out,
    test_size=0.25,random_state = 42)

In [ ]:
xg = XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.001)
# Fit the Model

xg.fit( im_train, out_train)
xg.score( im_test, out_test)



**LR**

In [ ]:
model = LogisticRegression(penalty = 'l2',
	  C = 1.0,
	  n_jobs = -1,
	  multi_class = 'auto',
	  solver = 'lbfgs')
model.fit(X, Y)

In [ ]:
explainer = shap.LinearExplainer(model,X)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, features=X, feature_names=X.columns)

In [ ]:
# top 10 important features
imp=da[["Age", "Intubation", "rate Po2", "Other chronic diseases",
       "sample for test", "Heart disease", "Cough",
       "Diabetes", "Shortness of breath",
       "gender","death"]]
imp

In [ ]:
lr_imp_out = imp['death']
lr_imp_input = imp[[i for i in imp.columns if i!='death' and imp[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
model = LogisticRegression(solver='newton-cg',multi_class='ovr',max_iter=100, fit_intercept= True)
model.fit(lr_imp_input, lr_imp_out)

In [ ]:
model.score(lr_imp_input, lr_imp_out)

In [ ]:
from sklearn.model_selection import train_test_split
im_train, im_test, out_train, out_test = train_test_split(
    lr_imp_input,
    lr_imp_out,
    test_size=0.25,random_state = 42)

In [ ]:
model = LogisticRegression(solver='newton-cg',multi_class='ovr',max_iter=100, fit_intercept= True)
model.fit( im_train, out_train)
model.score( im_test, out_test)

data slipt

In [ ]:
Y = da['death']
X = da[[i for i in da.columns if i!='death' and da[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

SVM

In [ ]:
# Build the SVM model
from sklearn import svm
svm = svm.SVC(gamma='scale',decision_function_shape='ovo')
svm.fit(X_train, Y_train)
# The SHAP values
svm_explainer = shap.KernelExplainer(svm.predict,X_test)
svm_shap_values = svm_explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(svm_shap_values, X_test)

In [ ]:
da = pd.read_csv("/content/drive/MyDrive/thesis/project/Gsesir_Copy.csv")
da

In [ ]:
# top 10 important features
imp=da[["Age", "Intubation", "rate Po2", "Immune Deficiency (Acquired or Congenital)",
       "HIV/AIDS", "Heart disease", "Liver disease",
       "Diabetes", "Shortness of breath",
       "gender","death"]]
imp

In [ ]:
svm_imp_out = imp['death']
svm_imp_input = imp[[i for i in imp.columns if i!='death' and imp[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
model = SVC(gamma='scale',decision_function_shape='ovo')
model.fit(svm_imp_input, svm_imp_out)
model.score(svm_imp_input, svm_imp_out)

KNN

In [ ]:
# Train the KNN model
from sklearn import neighbors
n_neighbors = 15
knn = neighbors.KNeighborsClassifier(n_neighbors,weights='distance')
knn.fit(X_train, Y_train)
# Produce the SHAP values
knn_explainer = shap.KernelExplainer(knn.predict,X_test)
knn_shap_values = knn_explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(knn_shap_values, X_test)

In [ ]:
da = pd.read_csv("/content/drive/MyDrive/thesis/project/Gsesir_Copy.csv")
da

In [ ]:
# top 10 important features
imp=da[["Age","hospital duration", "rate Po2", "Fever",
       "cantact coronadisease", "Heart disease", "Other chronic diseases",
       "Diabetes", "Shortness of breath",
       "gender","death"]]
imp

In [ ]:
knn_imp_out = imp['death']
knn_imp_input = imp[[i for i in imp.columns if i!='death' and imp[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
from sklearn import neighbors
n_neighbors = 15
knn = neighbors.KNeighborsClassifier(n_neighbors,weights='distance')
knn.fit(knn_imp_input, knn_imp_out)
knn.score(knn_imp_input, knn_imp_out)

Decision Tree

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, Y_train)
# Produce the SHAP values

dt_explainer = shap.KernelExplainer(dt.predict, X_test)
dt_shap_values = dt_explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(dt_shap_values, X_test)

In [ ]:
da = pd.read_csv("/content/drive/MyDrive/thesis/project/Gsesir_Copy.csv")
da

In [ ]:
# top 10 important features
imp=da[["Age","hospital duration", "rate Po2", "Fever",
       "cantact coronadisease", "Heart disease", "Other chronic diseases",
       "Loss of consciousness", "Shortness of breath",
       "myalgia or fatigue","death"]]
imp

In [ ]:
dt_imp_out = imp['death']
dt_imp_input = imp[[i for i in imp.columns if i!='death' and imp[i].dtype in [np.int64, np.float64, np.object]]]

In [ ]:
from sklearn.model_selection import train_test_split
im_train, im_test, out_train, out_test = train_test_split(
    dt_imp_input,
    dt_imp_out,
    test_size=0.20,random_state = 42)

In [ ]:
dt = DecisionTreeClassifier()

dt.fit(im_train, out_train)
dt.score(im_test,out_test)

Random Forest

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
random_forest_preds = random_forest.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,Y_test))

In [ ]:
rf_explainer = shap.TreeExplainer(random_forest)
rf_shap_values = rf_explainer.shap_values(X_test)


In [ ]:
shap.summary_plot(rf_shap_values, X_test)

#**importance Features find by shap accuracy test**

# Feature importance

In [ ]:
# selecting rows based on condition
age_to_34 = da.loc[da['Age'] <= 34]
  
age_to_34.sort_values("Age")

In [ ]:
# pandas count distinct values in column
age_to_34['death'].value_counts()

In [ ]:
Y = age_to_34['death']
X = age_to_34[[i for i in age_to_34.columns if i!='death' and age_to_34[i].dtype in [np.int64, np.float64, np.object]]]

**Random Forest**

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(rfc.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

**Decission Tree**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X, Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(dt.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

XGBoost

In [ ]:
xg = XGBClassifier(eta = 0.1,
	 max_depth = 10,
	 min_child_weight = 5,
	 n_estimators = 100,
	 n_jobs = 1)

xg.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(xg.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

XGB

In [ ]:
#X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=12)
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.25, random_state = 1)

In [ ]:
xgbr = XGBRegressor()
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
val_X_scaled = scaler.fit_transform(val_X)
val_X_mean = scaler.mean_
val_X_std = scaler.var_**(1/2)

In [ ]:
xgb_model = xgbr.fit(train_X_scaled,train_Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(xgbr.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)


In [ ]:
explainer = shap.TreeExplainer(xgb_model, val_X_scaled)
shap_values = explainer.shap_values(val_X_scaled)
shap.summary_plot(shap_values, val_X_scaled)

In [ ]:
shap.summary_plot(shap_values, val_X, plot_type="bar")

In [ ]:
shap.dependence_plot("Shortness of breath", shap_values,val_X)

Logistic Regression

In [ ]:
model = LogisticRegression(penalty = 'l2',
	  C = 1.0,
	  n_jobs = -1,
	  multi_class = 'auto',
	  solver = 'lbfgs')
model.fit(train_X, train_Y)

In [ ]:
explainer = shap.LinearExplainer(model,val_X)
shap_values = explainer.shap_values(val_X)
shap.summary_plot(shap_values, val_X)

SVC

In [ ]:
sc = StandardScaler()
sc.fit(train_X)
X_train_std = sc.transform(train_X)
X_test_std = sc.transform(val_X)

In [ ]:
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=1.0, random_state=1, kernel='linear')
 
# Fit the model
svc.fit(X_train_std, train_Y)

In [ ]:
subset_to_explain_scaled = X_test_std
explainer = shap.KernelExplainer(svc.predict, subset_to_explain_scaled)
shap_values = explainer.shap_values(subset_to_explain_scaled)
shap.summary_plot(shap_values, subset_to_explain_scaled)

LR

In [ ]:
lr = LogisticRegression(solver='newton-cg',multi_class='ovr',max_iter=100, fit_intercept= True)
lr.fit(train_X, train_Y)
lr.score(val_X, val_Y)

SVM

In [ ]:
svm = SVC(gamma='scale')
svm.fit(train_X, train_Y)
svm.score(val_X, val_Y)

RF

In [ ]:
rf = RandomForestClassifier(n_estimators=20)
rf.fit(train_X, train_Y)
rf.score(val_X, val_Y)

DT

XGBoost

In [ ]:
xgbr = XGBRegressor()
xgbr.fit(train_X, train_Y)
xgbr.score(val_X, val_Y)

# **After 34 age**

In [ ]:
# selecting rows based on condition
age_after_34 = da.loc[(da['Age'] > 34) & (da['Age']<=47)]
age_after_34.sort_values("Age")

In [ ]:
# pandas count distinct values in column
age_after_34['death'].value_counts()

In [ ]:
Y = age_after_34['death']
X = age_after_34[[i for i in age_after_34.columns if i!='death' and age_after_34[i].dtype in [np.int64, np.float64, np.object]]]

**Random** **Forest**

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(rfc.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

**Decision Tree**

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(dt.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

XGBoost Classifier

In [ ]:
xg = XGBClassifier(eta = 0.1,
	 max_depth = 10,
	 min_child_weight = 5,
	 n_estimators = 100,
	 n_jobs = 1)

xg.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(xg.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

Data Slipt

In [ ]:
#X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=12)
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.25, random_state = 1)

**LR**

In [ ]:
model = LogisticRegression(penalty = 'l2',
	  C = 1.0,
	  n_jobs = -1,
	  multi_class = 'auto',
	  solver = 'lbfgs')
model.fit(train_X, train_Y)

In [ ]:
explainer = shap.LinearExplainer(model,val_X)
shap_values = explainer.shap_values(val_X)
shap.summary_plot(shap_values, val_X)

**Support Vector classifier**

In [ ]:
sc = StandardScaler()
sc.fit(train_X)
X_train_std = sc.transform(train_X)
X_test_std = sc.transform(val_X)

In [ ]:
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=1.0, random_state=1, kernel='linear')
 
# Fit the model
svc.fit(X_train_std, train_Y)

In [ ]:
subset_to_explain_scaled = X_test_std
explainer = shap.KernelExplainer(svc.predict, subset_to_explain_scaled)
shap_values = explainer.shap_values(subset_to_explain_scaled)
shap.summary_plot(shap_values, subset_to_explain_scaled)

In [ ]:
# Make the predictions
y_pred = svc.predict(X_test_std)
 
# Measure the performance
print("Accuracy score %.3f" %metrics.accuracy_score(val_Y, y_pred))

# **After 47**

In [ ]:
# selecting rows based on condition
age_after_47 = da.loc[(da['Age'] > 47) & (da['Age']<=64)]
age_after_47.sort_values("Age")

In [ ]:
# pandas count distinct values in column
age_after_47['death'].value_counts()

In [ ]:
Y = age_after_47['death']
X = age_after_47[[i for i in age_after_47.columns if i!='death' and age_after_47[i].dtype in [np.int64, np.float64, np.object]]]

Random Forest

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(rfc.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

DT

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(dt.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

XGB

In [ ]:
xg = XGBClassifier(eta = 0.1,
	 max_depth = 10,
	 min_child_weight = 5,
	 n_estimators = 100,
	 n_jobs = 1)

xg.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(xg.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

Data Slipt

In [ ]:
#X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=12)
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.25, random_state = 1)

LR

In [ ]:
model = LogisticRegression(penalty = 'l2',
	  C = 1.0,
	  n_jobs = -1,
	  multi_class = 'auto',
	  solver = 'lbfgs')
model.fit(train_X, train_Y)

In [ ]:
explainer = shap.LinearExplainer(model,val_X)
shap_values = explainer.shap_values(val_X)
shap.summary_plot(shap_values, val_X)

SVC

In [ ]:
sc = StandardScaler()
sc.fit(train_X)
X_train_std = sc.transform(train_X)
X_test_std = sc.transform(val_X)

In [ ]:
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=1.0, random_state=1, kernel='linear')
 
# Fit the model
svc.fit(X_train_std, train_Y)

In [ ]:
# Make the predictions
y_pred = svc.predict(X_test_std)
 
# Measure the performance
print("Accuracy score %.3f" %metrics.accuracy_score(val_Y, y_pred))

In [ ]:
subset_to_explain_scaled = X_test_std
explainer = shap.KernelExplainer(svc.predict, subset_to_explain_scaled)
shap_values = explainer.shap_values(subset_to_explain_scaled)
shap.summary_plot(shap_values, subset_to_explain_scaled)

# **All age**

In [ ]:
# selecting rows based on condition
age_to_all = da.loc[da['Age'] > 64]
  
age_to_all.sort_values("Age")

In [ ]:
# pandas count distinct values in column
age_to_all['death'].value_counts()

In [ ]:
Y = age_to_all['death']
X = age_to_all[[i for i in age_to_all.columns if i!='death' and age_to_all[i].dtype in [np.int64, np.float64, np.object]]]

RF

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(rfc.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

DT

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(dt.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

XGB

In [ ]:
xg = XGBClassifier(eta = 0.1,
	 max_depth = 10,
	 min_child_weight = 5,
	 n_estimators = 100,
	 n_jobs = 1)

xg.fit(X,Y)

In [ ]:
feat_imp = pd.DataFrame({"Feature Name": X.columns , "Importance": list(xg.feature_importances_)})
feat_imp.sort_values(["Importance"], ascending= False)

Data slipt

In [ ]:
#X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=12)
train_X, val_X, train_Y, val_Y = train_test_split(X, Y, test_size=0.25, random_state = 1)

LR

In [ ]:
model = LogisticRegression(penalty = 'l2',
	  C = 1.0,
	  n_jobs = -1,
	  multi_class = 'auto',
	  solver = 'lbfgs')
model.fit(train_X, train_Y)

In [ ]:
explainer = shap.LinearExplainer(model,val_X)
shap_values = explainer.shap_values(val_X)
shap.summary_plot(shap_values, val_X)

SVC

In [ ]:
sc = StandardScaler()
sc.fit(train_X)
X_train_std = sc.transform(train_X)
X_test_std = sc.transform(val_X)

In [ ]:
# Instantiate the Support Vector Classifier (SVC)
svc = SVC(C=1.0, random_state=1, kernel='linear')
 
# Fit the model
svc.fit(X_train_std, train_Y)

In [ ]:
# Make the predictions
y_pred = svc.predict(X_test_std)
 
# Measure the performance
print("Accuracy score %.3f" %metrics.accuracy_score(val_Y, y_pred))

In [ ]:
subset_to_explain_scaled = X_test_std
explainer = shap.KernelExplainer(svc.predict, subset_to_explain_scaled)
shap_values = explainer.shap_values(subset_to_explain_scaled)
shap.summary_plot(shap_values, subset_to_explain_scaled)